## Ising Model GAN 

In [ ]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "..")
import time
from IPython import display
from model import RegressionModel
from pathlib import Path
data_path = Path("../../GetData/Python/Data")

In [ ]:
import tensorflow as tf

def train_step(batch, gen_loss_log, disc_loss_log, batch_size, noise_dim, generator, discriminator, generator_optimizer, discriminator_optimizer, regression_model):
      noise = tf.random.normal([batch_size, noise_dim])
      images, temps = batch
      with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        estimated_temps = regression_model(generated_images)
        real_output = discriminator(images, training=True)
        generated_output = discriminator(generated_images, training=True)
        gen_loss = generator_loss(generated_output, estimated_temps, temps)
        gen_loss_log.append(gen_loss)
        disc_loss = discriminator_loss(real_output, generated_output)
        disc_loss_log.append(disc_loss)
        
      gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
      gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
      
      generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
      discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def generator_loss(generated_image, estimated_temps, temps, lambda2 = 100):
    # todo L2, L3
    l1 = tf.reduce_mean(tf.losses.binary_crossentropy(tf.ones_like(generated_image), generated_image))
    l2 = tf.reduce_mean(tf.losses.mean_squared_error(estimated_temps, temps))
    return l1 + lambda2 * l2

def discriminator_loss(real_image, generated_image):
    real_loss = tf.losses.binary_crossentropy(tf.ones_like(real_image), real_image, from_logits = True)
    generated_loss = tf.losses.binary_crossentropy(tf.zeros_like(generated_image), generated_image, from_logits = True)
    total_loss = real_loss + generated_loss
    return tf.reduce_mean(total_loss)

def make_generator_model():
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Dense(4 * 4 * 1024, use_bias=False, input_shape=(100,)))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Reshape((4, 4, 1024)))
    assert model.output_shape == (None, 4, 4, 1024)
    
    model.add(tf.keras.layers.Conv2DTranspose(512, (1, 1), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None,  8, 8, 512)  
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 16, 16, 256)  
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 32, 32, 128)    
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Conv2DTranspose(1, (5, 5), strides=(1, 1), padding='same', use_bias=False, activation=tf.sigmoid))

    assert model.output_shape == (None, 32, 32, 1)
    return model

def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(32, (5, 5), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.5))
      
    model.add(tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.5))
       
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1))
     
    return model


In [ ]:
import tensorflow as tf
from input_pipeline import dataset_tfrecord_pipeline

class DataIterator:
    def __init__(self, datasets, temps):
        self.datasets = datasets
        self.temps = temps
    def __iter__(self):
        self.data_iterators = [iter(data) for data in self.datasets]
        return self
    
    def __next__(self):
        data_list = []
        temp_list = []
        for index, data_iterator in enumerate(self.data_iterators):
            data = next(data_iterator)
            data_list.append(data)
            temp = self.temps[index] * np.ones((data.shape[0], 1))
            temp_list.append(temp)
        temps = tf.concat(temp_list, axis=0)
        data = tf.concat(data_list, axis=0)
        return data, temps
        
def make_dataset(data_dir, temps, batch_size=100, flatten=False):
    if isinstance(data_dir, str):
        data_dir = Path(data_dir)
    
    assert batch_size % len(temps) == 0, "Batch size must be divisible by the number of temperatures"

    trainset = []
    testset = []
    for temp in temps:
        dataset = dataset_tfrecord_pipeline(data_dir / f"Data{temp:.1f}.tfrecord", flatten=flatten, batch_size=batch_size // len(temps))
        trainset.append(dataset)
        dataset = dataset_tfrecord_pipeline(data_dir / f"TestData{temp:.1f}.tfrecord", flatten=flatten, batch_size=batch_size // len(temps))
        testset.append(dataset)
    gen_test = DataIterator(testset, temps)
    test_dataset = tf.data.Dataset.from_generator(lambda: gen_test, output_signature = (tf.TensorSpec(shape=(None, 32, 32, 1), dtype=tf.float32), tf.TensorSpec(shape=(None, 1), dtype=tf.float32)))
    gen_train = DataIterator(trainset, temps)
    train_dataset = tf.data.Dataset.from_generator(lambda: gen_train, output_signature = (tf.TensorSpec(shape=(None, 32, 32, 1), dtype=tf.float32), tf.TensorSpec(shape=(None, 1), dtype=tf.float32)))
    return train_dataset, test_dataset

In [ ]:
temps = np.arange(2.0, 3.1, 0.1)
train_dataset, _ = make_dataset(data_path, temps, batch_size=110, flatten=False)

In [ ]:
from datetime import datetime
from pathlib import Path
# Get the current date and time
current_datetime = datetime.now()

# Format it to include date, hour, and minutes
formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M")

results = Path("results/" + formatted_datetime)
results.mkdir()

In [ ]:
def generate_and_save_images(model, epoch, test_input):
  predictions = tf.round(model(test_input, training=False))

  plt.figure(figsize=(4,4))
  
  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0])
      plt.axis('off')
        
  plt.savefig(results / 'image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

def train(dataset, epochs, batch_size, gen_loss_log, disc_loss_log, regression_model):  
  for epoch in range(epochs):
    start = time.time()

    for batch in tqdm(dataset): 
      train_step(
             batch, 
             gen_loss_log, 
             disc_loss_log, 
             batch_size, 
             noise_dim, 
             generator, 
             discriminator, 
             generator_optimizer, 
             discriminator_optimizer,
             regression_model)

    display.clear_output(wait=True) 
    generate_and_save_images(
      generator,
      epoch + 1,
      random_vector_for_generation
    ) 
    print (f"Time taken for epoch {epoch} is {time.time()- start} sec")
    

noise_dim = 100
num_examples_to_generate = 16
random_vector_for_generation = tf.random.normal([num_examples_to_generate,
                                                 noise_dim])
generator = make_generator_model()
discriminator = make_discriminator_model()
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)
regression_model = tf.keras.models.load_model("results/regression_model")

# Train

In [ ]:
for image, temp in train_dataset.take(1):
    print("Image shape:", image.shape)
    print("Temp shape:", temp.shape)

In [ ]:
EPOCHS=1
gen_loss_log=[]
disc_loss_log=[]
batch_size=110 

In [ ]:
%%time
train(train_dataset, EPOCHS, batch_size, gen_loss_log, disc_loss_log, regression_model)

## Plot the loss of the generator and discriminator

In [ ]:
fig, ax1 = plt.subplots()

ax1.plot(np.asarray(disc_loss_log), color='tab:red')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Discriminator Loss', color='tab:red')
ax1.tick_params(axis='y', labelcolor='tab:red')

ax2 = ax1.twinx()
ax2.plot(np.asarray(gen_loss_log), color='tab:blue')
ax2.set_ylabel('Generator Loss', color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')

plt.show()

In [ ]:
generator.load_weights('generator_model.keras')

In [ ]:
plt.figure(figsize=(10, 10))
predictions = generator(random_vector_for_generation, training=False)
for i in range(16):
    plt.subplot(4, 4, i+1)
    spins = np.random.binomial(1, predictions[i, :, :])
    plt.imshow(spins)
    plt.axis('off')
plt.show()